In [62]:
import h5py
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import sys
import time
import pandas as pd
import cv2
import os
import imageio
import matplotlib as mpl
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import shutil

from deepcell.utils.tracking_utils import load_trks

# Make mask .npy

### Functions

In [54]:
# Function - make dorsal ventral coordinates ==> d_x, d_y, v_x, v_y

def make_dorsal_ventral_coor(dorsal, ventral, skeleton_id):
    d_x = []
    d_y = []
    v_x = []
    v_y = []

    for each in dorsal[skeleton_id][:]:
        d_x.append(each[0]/12.4)
        d_y.append(each[1]/12.4)

    for each in ventral[skeleton_id][:]:
        v_x.append(each[0]/12.4)
        v_y.append(each[1]/12.4)
    
    return d_x, d_y, v_x, v_y

In [55]:
def make_mask(d_x, d_y, v_x, v_y, well, worm_index):
    polygon = []

    x_i = 0
    while x_i < len(d_x):
        polygon.append(d_x[x_i])
        polygon.append(d_y[x_i])
        x_i += 1

    y_i = len(v_x)-1
    while y_i >= 0:
        polygon.append(v_x[y_i])
        polygon.append(v_y[y_i])
        y_i -= 1

    width = 3600
    height = 3036

    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=worm_index, fill=worm_index)
    
    # Define well coordinates
    if well == 'D4':
        mask = np.array(img)[180:710, 160:690]  
    elif well == 'D3':
        mask = np.array(img)[180:710, 880:1410]  
    elif well == 'D2':
        mask = np.array(img)[180:710, 1600:2130]
    elif well == 'D1':
        mask = np.array(img)[180:710, 2320:2850]
    elif well == 'C4':
        mask = np.array(img)[895:1425, 160:690]
    elif well == 'C3':
        mask = np.array(img)[895:1425, 880:1410]
    elif well == 'C2':
        mask = np.array(img)[895:1425, 1600:2130]
    elif well == 'C1':
        mask = np.array(img)[895:1425, 2320:2850]
    elif well == 'B4':
        mask = np.array(img)[1620:2150, 160:690] 
    elif well == 'B3':
        mask = np.array(img)[1620:2150, 880:1410] 
    elif well == 'B2':
        mask = np.array(img)[1620:2150, 1600:2130] 
    elif well == 'B1':
        mask = np.array(img)[1620:2150, 2320:2850] 
    elif well == 'A4':
        mask = np.array(img)[2340:2870, 180:710]
    elif well == 'A3':
        mask = np.array(img)[2340:2870, 880:1410]
    elif well == 'A2':
        mask = np.array(img)[2340:2870, 1600:2130]
    elif well == 'A1':
        mask = np.array(img)[2340:2870, 2320:2850] 
        
    mask_cp = np.copy(mask)
    mask_cp = mask_cp.astype(np.uint16)
    if worm_index != 255:
        mask_cp[mask_cp >= 255] = worm_index
        
    return mask_cp

In [100]:
# function - create 9002*530*530 mask for each well

def make_well_mask(well='A4', dorsal=dorsal, ventral=ventral, well_dict=well_dict, filename=False, down_sampling=1, frame_start=0, frame_end=9001):  
    pd = traj.loc[well_dict[well], :]  # So here it is the position_index, not skel_id
    list_stack = []
    start_time = time.time()
    array_2d = np.zeros(shape=(530, 530), dtype=np.int16)
    
    print("--------------------%s------------------------" % well)

    ## Second, do frame by frame
    frame_index = frame_start
    while frame_index <= frame_end:  
        array_2d = np.zeros(shape=(530, 530), dtype=np.int16)
        skel_id_inframe = list(pd[pd["frame_number"] == frame_index]['skeleton_id'])
        worm_id_inframe = list(pd[pd["frame_number"] == frame_index]['worm_index_joined'])

        for i, skel_id in enumerate(skel_id_inframe):
            worm_index = worm_id_inframe[i]
            dx, dy, vx, vy = make_dorsal_ventral_coor(dorsal, ventral, skel_id)
            mask = make_mask(dx, dy, vx, vy, well, worm_index)
            array_2d = array_2d + mask

        list_stack.append(array_2d)

        if frame_index == frame_start:
            print("Starts at: ", frame_index)
        
        if frame_index%200 == 0:
            print("Finish frame_index: ", frame_index)
            print("--- %s seconds ---" % (time.time() - start_time))

        if frame_index == frame_end:
            print("Finish!")
            print("--- %s seconds ---" % (time.time() - start_time))

        frame_index += 1
    print("length of list_stack: ", len(list_stack))
    stacked_array = np.stack(list_stack, axis=0)
    stacked_array = stacked_array[:,:,:, np.newaxis]
    
    if filename:
        output_filename = filename
    else:
        output_filename = "worm_data/22956814/mask_%s.npy" % well
    
    if down_sampling != 1:
        stacked_array = stacked_array[frame_start:(frame_end+1):down_sampling, :, :, :]
    
    np.save(output_filename, stacked_array)
    
    return stacked_array

### Create variables

In [4]:
# Input file
fN = h5py.File("../22956814_featuresN.hdf5")

# Define dorsal and ventral coordinates
coordinates = fN['coordinates']
dorsal = coordinates['dorsal_contours']
ventral = coordinates['ventral_contours']

# trajectory_data and timeseries_data
traj = pd.read_hdf("../22956814_featuresN.hdf5", '/trajectories_data')
timeseries = fN['timeseries_data']

In [5]:
# Get wells - data from timeseries_data, at position [2]

wells = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'D4']
#wells = [each.encode() for each in wells]

well_dict = {}
for each in wells:
    well_dict[each] = []
    
#print('A1'.encode().decode())
for i, each in enumerate(timeseries[:]):  # "i" == index
    well = each[2].decode()
    if well in well_dict.keys():
        well_dict[well].append(i)

### Create mask .npy

In [74]:
import random

frame_starts = [random.randint(0, 8401) for x in range(5)]

In [105]:
frame_start = 0
down_sampling = 6
frame_end = frame_start+100*down_sampling-1

a1_m1 = make_well_mask('A1', down_sampling=down_sampling, frame_start=frame_start, frame_end=frame_end, filename='../22956814/a1_mask_1.npy')

--------------------A1------------------------
Starts at:  0
Finish frame_index:  0
--- 0.017968177795410156 seconds ---
Finish frame_index:  200
--- 2.8046185970306396 seconds ---
Finish frame_index:  400
--- 5.4634013175964355 seconds ---
Finish!
--- 8.242723941802979 seconds ---
length of list_stack:  600


In [106]:
a1_m1.shape

(100, 530, 530, 1)

In [67]:
# Check gif

def plot_gif(np_array, output_fn):
    mpl.rcParams['image.interpolation'] = 'none'  # Prevent mpl smoothes the edges
    os.mkdir("pic_temporary")  # To temporarily store the files

    #spring = cm.get_cmap('spring', 256)
    #newcolors = spring(np.linspace(0, 256, 256))
    #black = np.array([0/256, 0/256, 0/256, 1])
    #newcolors[:1, :] = black
    #newcmp = ListedColormap(newcolors)

    pic_list = []
    flag = 0

    for i in range(np_array.shape[0]):
        #plt.imshow(np_array[i], cmap=newcmp)
        mask = np_array[i]
        mask = np.where(mask==0, -1, mask) 
        #print(np.unique(mask_1))

        value = -1
        masked_array = np.ma.masked_where(mask == value, mask)

        #cmap = mpl.cm.get_cmap("spring")
        cmap = mpl.cm.get_cmap("spring").copy()
        cmap.set_bad(color='black')
        
        plt.imshow(masked_array, cmap=cmap)
        
        fn = 'pic_temporary/%d.jpg' % flag
        plt.savefig(fn)
        pic_list.append(fn)
        flag += 1
    
    with imageio.get_writer(output_fn, mode='I') as writer:
        for filename in pic_list:
            image = imageio.imread(filename)
            writer.append_data(image)
    
    # Remove the temporary file
    shutil.rmtree("pic_temporary")
    
    #return pic_list

In [ ]:
plot_gif(m_a1_0_600, "../22956814/mask_a1_0_600.gif")

# Make raw video npy

### Functions

In [ ]:
def get_raw_well(well, well_name='A4', filename=False, down_sampling=False, frame_start=0, frame_end=9001, down_sampling=1, filename=False):    
    print(well_name)
    frames = []
    
    ymin = well[0]
    ymax = well[1]
    xmin = well[2]
    xmax = well[3]

    path = "000000.mp4"
    cap = cv2.VideoCapture(path)
    ret = True
    while ret:
        ret, img = cap.read() # read one frame from the 'capture' object; img is (H, W, C)
        if ret:
            frames.append(img[ymin:ymax, xmin:xmax, :])
    
    stacked = np.stack(frames, axis=0)
    stacked = stacked.astype(dtype='float32')
    stacked_mean = np.mean(stacked, axis=3)[:,:,:,None]
    
    if frame_start != 0 or frame_end != 9001 or down_sampling != 1:
        stacked_mean = stacked_mean[frame_start:(frame_end+1):down_sampling, :, :, :]
    
    if filename:
        output_fn = filename
    else:
        output_fn = 'well_A/raw_%s.npy' % well_name
        
    np.save(output_fn, stacked_mean)
    
    return stacked_mean

### Define well regions

In [ ]:
# get well by well, and store
d4 = [180, 710, 160, 690]
d3 = [180, 710, 880, 1410]
d2 = [180, 710, 1600, 2130]
d1 = [180, 710, 2320, 2850]

c4 = [895, 1425, 160, 690]
c3 = [895, 1425, 880, 1410]
c2 = [895, 1425, 1600, 2130]
c1 = [895, 1425, 2320, 2850]

b4 = [1620, 2150, 160, 690]
b3 = [1620, 2150, 880, 1410]
b2 = [1620, 2150, 1600, 2130]
b1 = [1620, 2150, 2320, 2850]

a4 = [2340, 2870, 160, 690]
a3 = [2340, 2870, 880, 1410]
a2 = [2340, 2870, 1600, 2130]
a1 = [2340, 2870, 2320, 2850]

In [ ]:
get_raw_well(a1, "A1", small=True, down_sampling=False, frame_start=0, frame_end=599, down_sampling=6, filename="22956814/A1_raw_0_599.npy")
#get_raw_well(a1, "A1", small=True, down_sampling=False, frame_start=600, frame_end=1199, down_sampling=6, filename="22956814/A1_raw_600_1199.npy")
#get_raw_well(a1, "A1", small=True, down_sampling=False, frame_start=1200, frame_end=1799, down_sampling=6, filename="22956814/A1_raw_1200_1799.npy")
#get_raw_well(a1, "A1", small=True, down_sampling=False, frame_start=1800, frame_end=2399, down_sampling=6, filename="22956814/A1_raw_1800_2399.npy")
#get_raw_well(a1, "A1", small=True, down_sampling=False, frame_start=2400, frame_end=2999, down_sampling=6, filename="22956814/A1_raw_2400_2999.npy")

### Create raw .npy files

# Make lineages dict

In [48]:
def make_lineages(well, frame_start=0, frame_end=9002):  # TODO: check the unique worm ids within the frame_start and frame_end
    pd= traj.loc[well_dict[well], :]
    worm_ids = pd['worm_index_joined']
    unique_worm_ids = list(np.unique(worm_ids))  
    
    lineages = {}
        
    for worm in unique_worm_ids:
        worm_pd = pd.loc[pd['worm_index_joined'] == worm]
        worm_pd_trim = worm_pd[worm_pd['frame_number'].between(frame_start, frame_end)]
        
        if worm_pd_trim.empty == False:  # So only adds in the worm in the frames    
            lineages[int(worm)] = {}
            
            nested_dict = {}
            nested_dict['label'] = int(worm)
            
            frame_list = []
            for frame in worm_pd_trim['frame_number']:  # TODO: edit here
                frame_list.append(int(frame))
            nested_dict['frames'] = frame_list

            nested_dict['daughters'] = []
            nested_dict['capped'] = False
            nested_dict['frame-div'] = None
            nested_dict['parent'] = None

            lineages[worm] = nested_dict
        
    return lineages